In [7]:
# Importing necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import random as rn
import joblib
from torch.utils.data import Dataset, DataLoader
import time
import torch.nn.functional as F

In [8]:
# Preset parameters
RANDOM_SEED = 42
VALIDATE_SIZE = 0.2

In [9]:
# Setting random seeds to ensure reproducibility
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
rn.seed(RANDOM_SEED)

In [10]:
# Patient file paths
patients = {
    1: ['Data/organized_fcs_data1a.csv', 'Data/organized_fcs_data1b.csv', 'Data/organized_fcs_data1c.csv'],
    2: ['Data/organized_fcs_data2a.csv', 'Data/organized_fcs_data2b.csv', 'Data/organized_fcs_data2c.csv'],
    3: ['Data/organized_fcs_data3a.csv', 'Data/organized_fcs_data3b.csv', 'Data/organized_fcs_data3c.csv'],
    4: ['Data/organized_fcs_data4a.csv', 'Data/organized_fcs_data4b.csv', 'Data/organized_fcs_data4c.csv'],
    5: ['Data/organized_fcs_data5a.csv', 'Data/organized_fcs_data5b.csv', 'Data/organized_fcs_data5c.csv'],
    6: ['Data/organized_fcs_data6a.csv', 'Data/organized_fcs_data6b.csv', 'Data/organized_fcs_data6c.csv'],
    7: ['Data/organized_fcs_data7.csv'],
    8: ['Data/organized_fcs_data8a.csv', 'Data/organized_fcs_data8b.csv', 'Data/organized_fcs_data8c.csv'],
    9: ['Data/organized_fcs_data9a.csv', 'Data/organized_fcs_data9b.csv'],
    10: ['Data/organized_fcs_data10a.csv', 'Data/organized_fcs_data10b.csv', 'Data/organized_fcs_data10c.csv'],
    11: ['Data/organized_fcs_data11.csv'],
    12: ['Data/organized_fcs_data12a.csv', 'Data/organized_fcs_data12b.csv', 'Data/organized_fcs_data12c.csv']
}

In [11]:
training_dfs = []
testing_dfs = []

first_six_ids = [1, 2, 3, 4, 5, 6]
training_ids = [1,2,3,4,6]     #leaving out patient no. 5 for validation

for patient_id, file_paths in patients.items():
    patient_data = pd.concat([pd.read_csv(file).drop(columns=['Time'], errors='ignore') for file in file_paths])
    
    if patient_id in training_ids:
        training_dfs.append(patient_data)
    if patient_id not in training_ids and patient_id in first_six_ids:
        testing_dfs.append(patient_data)

train_full = pd.concat(training_dfs, ignore_index=True)
test_full = pd.concat(testing_dfs, ignore_index=True)

In [12]:
print(training_ids)

[1, 2, 3, 4, 6]


In [13]:
print(f"Training data samples: {len(training_dfs)}")
print(f"Testing data samples: {len(testing_dfs)}")

Training data samples: 5
Testing data samples: 1


In [14]:
pipeline = Pipeline([('scaler', MinMaxScaler())])
pipeline.fit(train_full)

Pipeline(steps=[('scaler', MinMaxScaler())])

In [15]:
# transform the training and validation data with these parameters
X_train_transformed = pipeline.transform(train_full)
X_validate_transformed = pipeline.transform(test_full)

In [16]:
input_dim = X_train_transformed.shape[1]
BATCH_SIZE = 256
EPOCHS = 100

In [17]:
print(input_dim)

14


In [18]:
class CustomDataset(Dataset):
    def __init__(self,dataset):
        self.dataset = torch.tensor(dataset, dtype=torch.float32)
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        return self.dataset[idx]

In [19]:
train_dataset = CustomDataset(X_train_transformed)
val_dataset = CustomDataset(X_validate_transformed)

In [20]:
train_loader = DataLoader(train_dataset, BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, BATCH_SIZE, shuffle=False)

In [21]:
class VarAutoEncoder(nn.Module):
    def __init__(self, input_dim, latent_dim=2):
        super().__init__()
        # Encoder layers
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ELU(),
            nn.Linear(16, 8),
            nn.ELU(),
            nn.Linear(8, 4),
            nn.ELU(),
        )

        self.fc_mu = nn.Linear(4, latent_dim)
        self.fc_log_var = nn.Linear(4, latent_dim)

        # Decoder layers
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 4),
            nn.ELU(),
            nn.Linear(4, 8),
            nn.ELU(),
            nn.Linear(8, 16),
            nn.ELU(),
            nn.Linear(16, input_dim),
            nn.ELU()
        )

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        encoded = self.encoder(x)
        mu = self.fc_mu(encoded)
        log_var = self.fc_log_var(encoded)
        z = self.reparameterize(mu, log_var)
        decoded = self.decoder(z)
        return decoded, mu, log_var

In [22]:
autoencoder= VarAutoEncoder(input_dim=input_dim)
print(autoencoder)

VarAutoEncoder(
  (encoder): Sequential(
    (0): Linear(in_features=14, out_features=16, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=16, out_features=8, bias=True)
    (3): ELU(alpha=1.0)
    (4): Linear(in_features=8, out_features=4, bias=True)
    (5): ELU(alpha=1.0)
  )
  (fc_mu): Linear(in_features=4, out_features=2, bias=True)
  (fc_log_var): Linear(in_features=4, out_features=2, bias=True)
  (decoder): Sequential(
    (0): Linear(in_features=2, out_features=4, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=4, out_features=8, bias=True)
    (3): ELU(alpha=1.0)
    (4): Linear(in_features=8, out_features=16, bias=True)
    (5): ELU(alpha=1.0)
    (6): Linear(in_features=16, out_features=14, bias=True)
    (7): ELU(alpha=1.0)
  )
)


In [23]:
def vae_loss_function(reconstructed, original, mu, log_var, beta=1):
    recon_loss = F.mse_loss(reconstructed, original, reduction='mean')
    kl_loss = -0.5 * torch.mean(1 + log_var - mu.pow(2) - log_var.exp())
    return recon_loss + beta * kl_loss

In [24]:
def train_autoencoder(model, train_loader, val_loader, epochs=100, lr=0.001):
    device = torch.device('cpu')
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    patience = 15
    patience_counter = 0
    
    print(f"Training Variational Autoencoder for {epochs} epochs...")
    print("-" * 50)
    
    for epoch in range(epochs):
        start_time = time.time()
        model.train()
        train_loss = 0.0
        for batch_data in train_loader:
            batch_data = batch_data.to(device)
            optimizer.zero_grad()
            reconstructed, mu, log_var = model(batch_data)
            loss = vae_loss_function(reconstructed, batch_data, mu, log_var, beta=1)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        avg_train_loss = train_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch_data in val_loader:
                batch_data = batch_data.to(device)
                reconstructed, mu, log_var = model(batch_data)
                loss = vae_loss_function(reconstructed, batch_data, mu, log_var, beta=1)
                val_loss += loss.item()
        avg_val_loss = val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            best_model = model.state_dict().copy()
        else:
            patience_counter += 1
        
        epoch_time = time.time() - start_time
        print(f"Epoch {epoch+1:3d}/{epochs} | Train: {avg_train_loss:.6f} | Val: {avg_val_loss:.6f} | Time: {epoch_time:.1f}s | Patience: {patience_counter}/{patience}")
        if patience_counter >= patience:
            print(f"\nEarly stopping at epoch {epoch+1}")
            print(f"Best validation loss: {best_val_loss:.6f}")
            model.load_state_dict(best_model)
            break

    print("Training completed!")
    return model, train_losses, val_losses

In [25]:
def run_training():
    vae = autoencoder
    print(f"Model created with {sum(p.numel() for p in vae.parameters())} parameters")
    trained_model, train_history, val_history = train_autoencoder(
        model=vae,
        train_loader=train_loader,
        val_loader=val_loader,
        epochs=EPOCHS,
        lr=0.001
    ) 
    return trained_model, train_history, val_history

if __name__ == "__main__":
    trained_model, train_losses, val_losses = run_training()

Model created with 866 parameters
Training Variational Autoencoder for 100 epochs...
--------------------------------------------------
Epoch   1/100 | Train: 0.016839 | Val: 0.011610 | Time: 188.1s | Patience: 0/15
Epoch   2/100 | Train: 0.016783 | Val: 0.011645 | Time: 126.4s | Patience: 1/15
Epoch   3/100 | Train: 0.016781 | Val: 0.011696 | Time: 115.8s | Patience: 2/15
Epoch   4/100 | Train: 0.016781 | Val: 0.011693 | Time: 115.9s | Patience: 3/15
Epoch   5/100 | Train: 0.016780 | Val: 0.011732 | Time: 124.5s | Patience: 4/15
Epoch   6/100 | Train: 0.016780 | Val: 0.011641 | Time: 127.1s | Patience: 5/15
Epoch   7/100 | Train: 0.016780 | Val: 0.011589 | Time: 126.1s | Patience: 0/15
Epoch   8/100 | Train: 0.016780 | Val: 0.011655 | Time: 115.6s | Patience: 1/15
Epoch   9/100 | Train: 0.016780 | Val: 0.011593 | Time: 114.4s | Patience: 2/15
Epoch  10/100 | Train: 0.016780 | Val: 0.011723 | Time: 127.6s | Patience: 3/15
Epoch  11/100 | Train: 0.016780 | Val: 0.011617 | Time: 116.5s |

In [26]:
#Saving Only the model weights
torch.save(trained_model.state_dict(), 'vae_2dim_b1_LOPO.pth')

# Loading the model weights
# autoencoder = VarAutoEncoder(input_dim=input_dim, latent_dim=2)
# autoencoder.load_state_dict(torch.load("vae_2_LOPO.pth"))
# autoencoder.eval()

In [27]:
def test_autoencoder_on_patient(patient_id, model, pipeline, patients_dict):
    file_paths = patients_dict[patient_id]
    patient_df = pd.concat([pd.read_csv(fp).drop(columns=['Time'], errors='ignore') for fp in file_paths], ignore_index=True)
    patient_data_transformed = pipeline.transform(patient_df)
    patient_tensor = torch.tensor(patient_data_transformed, dtype=torch.float32)
    model.eval()
    with torch.no_grad():
        reconstructed, _, _ = model(patient_tensor)
        reconstructed = reconstructed.numpy()
    mse = np.mean((patient_data_transformed - reconstructed) ** 2, axis=1)
    return mse


In [52]:
pt1 = test_autoencoder_on_patient(5, trained_model, pipeline, patients)
print(pt1)

[0.00675749 0.00346686 0.00521256 ... 0.00258324 0.01831031 0.0059909 ]


In [ ]:
#      Predicted      Actual.     Reconstruction Error
# 7    3.92%          3.28%         0.06049
# 8    0.40%          1.2%          0.04856
# 9    2.49%          9.3%          0.04527
# 10   3.17%          2.17%         0.06527
# 11   14.84%         14.6%         0.05654
# 12   4.63%          4.2%          0.05897



#Avg threshold = 0.055850000000000004


# For Healthy patient 5 --> 0.23%

In [39]:
percent_unhealthy = 4.2
threshold_index = int((percent_unhealthy / 100) * len(pt1))
threshold_value = np.sort(pt1)[-threshold_index]

print(f"Threshold for top {percent_unhealthy}% reconstruction errors: {threshold_value:.5f}")

Threshold for top 4.2% reconstruction errors: 0.05897


In [40]:
print((0.06049+0.04856+0.04527+0.06527+0.05654+0.05897)/6)

0.055850000000000004


In [53]:
threshold = 0.055850000000000004

# Calculate percentage above threshold
num_above = np.sum(pt1 > threshold)
total = len(pt1)
percent_above = (num_above / total) * 100

print(f"{percent_above:.2f}% of cells have reconstruction error > {threshold}")

0.23% of cells have reconstruction error > 0.055850000000000004
